In [2]:
from functools import partial
from typing import List
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import brute
import cirq
import openfermion as of
from openfermionpyscf import generate_molecular_hamiltonian

from mitiq import PauliString, Observable, Executor, zne  # Zero-noise extrapolation module
from mitiq.interface.mitiq_cirq import compute_density_matrix

In [4]:
radii = [0.2 * i for i in range(1, 14)]

def qubit_operator_to_pauli_sum(qubit_op) -> List:
    """Converts the OpenFermion qubit operator to a list of mitiq Pauli Strings."""
    psum = []
    for ind_ops, coeff in qubit_op.terms.items():
        if ind_ops == tuple():
            psum.append(PauliString("I", coeff))
            continue
        term = ["I", "I"]
        for ind, op in ind_ops:
            term[ind] = op
        term = ''.join(term)
        psum.append(PauliString(term, coeff))
    return psum

def get_hamiltonian(bond_length=0.7414) -> Observable:
    """Returns the hamiltonian for the H2 molecule at the given bond length. Uses the STO-6G basis set and the Bravyi-Kitaev
    transform to map fermionic operators to qubits. Results in a 2 qubit hamiltonian returned via a metriq.Observable."""
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    molecular_hamiltonian = generate_molecular_hamiltonian(geometry, basis='sto-6g', multiplicity=1, charge=0)
    fermionic_op = of.transforms.get_fermion_operator(molecular_hamiltonian)
    bk_op = of.transforms.symmetry_conserving_bravyi_kitaev(fermionic_op, active_orbitals=4, active_fermions=2)
    qubit_operator = qubit_operator_to_pauli_sum(bk_op)
    return Observable(*qubit_operator)

hamiltonians = get_hamiltonian(0.7414)
print(hamiltonians)

(-0.3477762456868043+0j)*I + (0.3943982566281438+0j)*Z(q(0)) + (0.39439825662814376+0j)*Z(q(1)) + (0.011280181225500119+0j)*Z(q(0))*Z(q(1)) + (0.18157637663956305+0j)*X(q(0))*X(q(1))


In [5]:
def ansatz(theta: float) -> cirq.Circuit:
    """Returns the circuit associated to the input variational parameter."""
    qreg = cirq.LineQubit.range(2)
    return cirq.Circuit(
        cirq.ops.ry(np.pi / 2).on(qreg[0]),
        cirq.ops.rx(-np.pi / 2).on(qreg[1]),
        cirq.ops.CNOT.on(*qreg),
        cirq.ops.rz(theta).on(qreg[1]),
        cirq.ops.CNOT.on(*qreg),
        cirq.ops.ry(-np.pi / 2).on(qreg[0]),
        cirq.ops.rx(np.pi / 2).on(qreg[1]),
    )

In [6]:
# Noise levels
pvals = (0.00, 0.02, 0.04)
# Variational parameters
thetas = np.linspace(0.0, 2.0 * np.pi, 10)

all_energies = []
for pval in pvals:
    energies = []
    for theta in thetas:
        results = hamiltonians.expectation(ansatz(theta), execute=partial(compute_density_matrix, noise_model_function=cirq.depolarize, noise_level=(pval,)))
        energies.append(np.real(results))
    all_energies.append(energies)

In [7]:
all_energies

[[0.45230039954185486,
  0.3844721019268036,
  -0.02070513367652893,
  -0.5736445784568787,
  -1.015619158744812,
  -1.1398247480392456,
  -0.8881438970565796,
  -0.3783407211303711,
  0.15104205906391144,
  0.45230039954185486],
 [0.3136563003063202,
  0.25243714451789856,
  -0.08560022711753845,
  -0.5422846078872681,
  -0.9039279222488403,
  -1.00131356716156,
  -0.7888737320899963,
  -0.3660111427307129,
  0.06941193342208862,
  0.3136563003063202],
 [0.19620239734649658,
  0.141729936003685,
  -0.13873636722564697,
  -0.5139639377593994,
  -0.8083789348602295,
  -0.8842217326164246,
  -0.7060047388076782,
  -0.35711705684661865,
  -0.0008078664541244507,
  0.19620239734649658]]

In [8]:
# Set noise scaling method
scaling_function = zne.scaling.fold_global
# Set extrapolation method
fac = zne.inference.RichardsonFactory(scale_factors=[1, 3, 5])
# Set number of trials per expectation value
num_to_average = 1

# To reproduce the results of the Mitiq paper use the following settings
# scaling_function = zne.scaling.fold_gates_at_random
# pfac = zne.inference.PolyFactory(order=3,
#                           scale_factors=[1., 1.5, 2., 2.5, 3., 3.5, 4., 4.5, 5., 5.5, 6.])
# num_to_average = 5
# pvals = (0.00, 0.02, 0.04, 0.06)
# thetas = np.linspace(0.0, 2.0 * np.pi, 20)

all_mitigated = []
for p in pvals[1:]:
    mitigated = []
    for theta in thetas:
        # Define an executor function
        executor = Executor(partial(compute_density_matrix, noise_model_function=cirq.depolarize, noise_level=(p,)))
        # Run ZNE
        zne_value = zne.execute_with_zne(
            ansatz(theta),
            executor,
            hamiltonians,
            factory=fac,
            scale_noise=scaling_function,
            num_to_average=num_to_average,
        )
        mitigated.append(np.real(zne_value))
    all_mitigated.append(mitigated)

In [9]:
all_mitigated

[[0.44314899295568394,
  0.37402514927089137,
  -0.027975210919976488,
  -0.5751027651131151,
  -1.009836971759795,
  -1.1299246549606317,
  -0.8795243203639977,
  -0.3742888867855072,
  0.14821302890777543,
  0.44314899295568394],
 [0.40127465128898493,
  0.3320577200502145,
  -0.051575329154730315,
  -0.5711715146899218,
  -0.9788302518427361,
  -1.0875334367156015,
  -0.8474586978554713,
  -0.36617030203342443,
  0.1274017263203852,
  0.40127465128898493]]